# Unitary ansatz entering the VQE

The final energy output of a VQE calculation will crucially depend on the ansatz/form of the parameterized unitary $\hat U(\boldsymbol{\theta})$ employed in state preparation. Here we review two popular approaches, the unitary coupled cluster and qubit coupled cluster methodologies, and benchmark them for energy calculations of small molecules.

In [1]:
import numpy as np
import tequila as tq
from utility import *
threshold = 1e-6 #Cutoff for UCC MP2 amplitudes and QCC ranking gradients

## Unitary Coupled Cluster (UCC)

The UCC ansatz is obtained by 'unitarizing' the traditional coupled cluster ansatz,
$$ e^{\hat T} \rightarrow e^{\hat T - \hat T^\dagger} \equiv \hat U_{\text{UCC}}$$


Due to non-commutativity of terms in $\hat T - \hat T^\dagger$, the UCC ansatz does not have a straightforward decomposition in terms of circuit primitives implementable on the quantum computer. Therefore, to obtain a form which can be compiled, we employ the Trotter approximation. The accuracy of the circuit ansatz relative to the exact UCC operator will be dependent on how many Trotter steps are employed. The number of Trotter steps is commonly set to its minimal value of one to avoid excessive circuit depth.

In [2]:
trotter_steps = 1

### H2 in STO-3G basis

Below is a sample VQE simulation using the UCCSD ansatz compiled using a single trotter step for H$_2$ in minimal basis at $R=2.5$ (Angstrom). For comparison, we can run FCI to obtain the true ground state energy.


In [3]:
xyz_data = get_molecular_data('h2', geometry=2.5, xyz_format=True)
basis='sto-3g'

h2 = tq.quantumchemistry.Molecule(geometry=xyz_data, basis_set=basis)

print('Number of spin-orbitals (qubits): {} \n'.format(2*h2.n_orbitals))

E_FCI = h2.compute_energy(method='fci')

print('FCI energy: {}'.format(E_FCI))

Number of spin-orbitals (qubits): 4 

FCI energy: -0.9360549199436612


The UCCSD VQE optimization is of the form
$$E = \min_{\boldsymbol{\theta}} \langle \text{HF} | \hat U_{\text{UCC}}^\dagger(\boldsymbol{\theta}) \hat H  \hat U_{\text{UCC}} (\boldsymbol{\theta}) | \text{HF} \rangle $$
To expedite the optimization process, we can set the initial guess of the amplitudes to zero, i.e. the optimization will begin at the Hartree Fock state. This heuristic is best suited for when Hartree Fock is believed to be in qualitative agreement with the true ground state. To further alleviate quantum resources, we can estimate the amplitudes using classical electronic structure methods (here, MP2 perturbation theory), and only include the unitaries with non-zero estimated amplitudes.

In [4]:
H = h2.make_hamiltonian()

print("\nHamiltonian has {} terms\n".format(len(H)))

U_UCCSD = h2.make_uccsd_ansatz(initial_amplitudes='MP2',threshold=threshold, trotter_steps=trotter_steps)

E = tq.ExpectationValue(H=H, U=U_UCCSD)

print('\nNumber of UCCSD amplitudes: {} \n'.format(len(E.extract_variables())))

print('\nStarting optimization:\n')

result = tq.minimize(objective=E, method="BFGS", initial_values={k:0.0 for k in E.extract_variables()}, tol=1e-6)

print('\nObtained UCCSD energy: {}'.format(result.energy))


Hamiltonian has 15 terms


Number of UCCSD amplitudes: 1 


Starting optimization:

Optimizer: <class 'tequila.optimizers.optimizer_scipy.OptimizerSciPy'> 
backend         : qulacs
samples         : None
save_history    : True
noise           : None

Method          : BFGS
Objective       : 1 expectationvalues
gradient        : 32 expectationvalues

active variables : 1

E=-0.70294360  angles= {(1, 0, 1, 0): 0.0}  samples= None
E=-0.92698008  angles= {(1, 0, 1, 0): -0.5644201636314392}  samples= None
E=-0.93354955  angles= {(1, 0, 1, 0): -0.756477668453395}  samples= None
E=-0.93605486  angles= {(1, 0, 1, 0): -0.6900732739717247}  samples= None
E=-0.93605492  angles= {(1, 0, 1, 0): -0.6904080632337539}  samples= None
E=-0.93605492  angles= {(1, 0, 1, 0): -0.6904070264134937}  samples= None
Optimization terminated successfully.
         Current function value: -0.936055
         Iterations: 5
         Function evaluations: 6
         Gradient evaluations: 6

Obtained UCCSD energy: -0.9

We see that the converged UCCSD energy is in exact agreement with the FCI energy, as expected for a $2$-electron system. 

### H2O in 6-31G basis

Now let us try a larger problem of H$_2$O in 6-31G basis. However, we will restrict the active space. The unrestricted problem leads to a $14$-qubit Hamiltonian, and $34$ UCCSD amplitudes to optimize even after neglecting the zero MP2 amplitudes. Therefore, we will remove some orbital degrees of freedom which are less important in accurately describing the electronic structure. By freezing all orbitals other than $0b_1$, $1b_1$, $2a_1$, and $3a_1$, we reduce the problem to an $8$-qubit Hamiltonian with $8$ UCCSD variational amplitudes.

In [5]:
xyz_data = get_molecular_data('h2o', geometry=1, xyz_format=True)

basis = '6-31g'
active = {'B1':[0,1], 'A1':[2,3]}
h2o = tq.quantumchemistry.Molecule(geometry=xyz_data, basis_set = basis, active_orbitals = active)

print('Number of spin-orbitals (qubits): {} \n'.format(2*h2o.n_orbitals))

E_FCI = h2o.compute_energy(method='fci')

print('FCI energy: {}'.format(E_FCI))

There are known issues with some psi4 methods and frozen virtual orbitals. Proceed with fingers crossed for hf.
Number of spin-orbitals (qubits): 8 

There are known issues with some psi4 methods and frozen virtual orbitals. Proceed with fingers crossed for fci.
FCI energy: -75.98980140961197


We will then run the UCCSD VQE simulation (***warning: tq.minimize will take several minutes - 1 hour + to converge for a VQE instance of this size.*** Smaller active spaces can be employed to lower VQE simulation runtimes).

In [6]:
H = h2o.make_hamiltonian()

print("\nHamiltonian has {} terms\n".format(len(H)))

U_UCCSD = h2o.make_uccsd_ansatz(initial_amplitudes='MP2',threshold=threshold, trotter_steps=trotter_steps)

E = tq.ExpectationValue(H=H, U=U_UCCSD)

print('\nNumber of UCCSD amplitudes: {} \n'.format(len(E.extract_variables())))

print('\nStarting optimization:\n')

result = tq.minimize(objective=E, method="BFGS", initial_values={k:0.0 for k in E.extract_variables()}, tol=1e-4)

print('\nObtained UCCSD energy: {}'.format(result.energy))


Hamiltonian has 185 terms


Number of UCCSD amplitudes: 8 


Starting optimization:

Optimizer: <class 'tequila.optimizers.optimizer_scipy.OptimizerSciPy'> 
backend         : qulacs
samples         : None
save_history    : True
noise           : None

Method          : BFGS
Objective       : 1 expectationvalues
gradient        : 512 expectationvalues

active variables : 8

E=-75.98071144  angles= {(3, 0, 3, 0): 0.0, (3, 1, 2, 0): 0.0, (2, 1, 2, 1): 0.0, (2, 0, 3, 1): 0.0, (3, 1, 3, 1): 0.0, (3, 0, 2, 1): 0.0, (2, 1, 3, 0): 0.0, (2, 0, 2, 0): 0.0}  samples= None
E=-75.57991345  angles= {(3, 0, 3, 0): -0.1516876220703125, (3, 1, 2, 0): -0.04468536376953125, (2, 1, 2, 1): -0.06198883056640625, (2, 0, 3, 1): -0.04468536376953125, (3, 1, 3, 1): -0.0497894287109375, (3, 0, 2, 1): 0.15348052978515625, (2, 1, 3, 0): 0.15348052978515625, (2, 0, 2, 0): -0.06015777587890625}  samples= None
E=-75.98420833  angles= {(3, 0, 3, 0): -0.012046029637687056, (3, 1, 2, 0): -0.003548616617439548, (2, 1, 2

We obtain a UCCSD energy within chemical accuracy (< 1.6 miliHartree from FCI energy) for the H$_2$O molecule near equilibrium geometry. As the bond distance approaches the dissociation limit, the energy deviation from FCI is typically expected to increase as electronic correlations increase during covalent bond-breaking. Furthermore, as HF becomes energetically more distant from the FCI wavefunction, the initial guess of all amplitudes being zero may lead to a local minimum. One can then instead initialize the amplitudes using random guesses, and repeat for $n$ samples to attempt to find the global minimum. To facilitate this, one can run the commented lines below. This will execute $n$ instances of tq.minimize for uniformly random initial parameters in interval $[0, 2\pi]$ and return the lowest optimization result.

In [7]:
#n=10
#result = minimize_E_random_guesses(objective=E, method='BFGS', tol=1e-4, n=n)

## Qubit Coupled Cluster (QCC)

In contrast to UCC, the QCC methodology makes no direct reference to fermionic algebra and seeks to construct an efficient ansatz directly in qubit-space by finding multi-qubit Pauli strings (entanglers) which lower energy. This is done through an energy-lowering heuristic employing the energy gradient with respect to a Pauli strings variational amplitude. As opposed to UCCSD, the circuit depth and number of parameter is chosen to meet hardware limitations, i.e. one must choose how many exponentiated Pauli strings will be entering the QCC ansatz.

### H2 in STO-3G basis

Below we perform the entangler screening protocol for H2 in minimal basis, and obtain one grouping of entanglers with non-zero energy gradient. We then select one of them to be used in the QCC VQE simulation.

In [8]:
xyz_data = get_molecular_data('h2', geometry=2.5, xyz_format=True)
basis='sto-3g'

h2 = tq.quantumchemistry.Molecule(geometry=xyz_data, basis_set='sto-3g')

hf_reference = hf_occ(2*h2.n_orbitals, h2.n_electrons)

H = h2.make_hamiltonian()

print("\nHamiltonian has {} terms\n".format(len(H)))

#Define number of entanglers to enter ansatz
n_ents = 1

#Rank entanglers using energy gradient criterion
ranked_entangler_groupings = generate_QCC_gradient_groupings(H.to_openfermion(), 
                                                             2*h2.n_orbitals, 
                                                             hf_reference, 
                                                             cutoff=threshold)

print('Grouping gradient magnitudes (Grouping : Gradient magnitude):')
for i in range(len(ranked_entangler_groupings)):
    print('{} : {}'.format(i+1,ranked_entangler_groupings[i][1]))


entanglers = get_QCC_entanglers(ranked_entangler_groupings, n_ents, 2*h2.n_orbitals)

print('\nSelected entanglers:')
for ent in entanglers:
    print(ent)




Hamiltonian has 15 terms

Grouping gradient magnitudes (Grouping : Gradient magnitude):
1 : 0.2822

Selected entanglers:
1.0 [X0 Y1 X2 X3]


Once the QCC ranking procedure has been ran, we can simulate the QCC VQE optimization with the generated entanglers. The VQE optimization for the QCC ansatz is of the form
$$E = \min_{\boldsymbol{\Omega}, \boldsymbol{\tau}} \langle \boldsymbol{\Omega} | U_{\text{ENT}}^\dagger (\boldsymbol{\tau}) \hat H  U_{\text{ENT}} (\boldsymbol{\tau}) | \boldsymbol{\Omega} \rangle $$
where $\boldsymbol{\Omega}$ denote collective Euler angles parameterizing single-qubit rotations, and $\boldsymbol{\tau}$ are entangler amplitudes. 

In [9]:
#Mean-field part of U (Omega):    
U_MF = construct_QMF_ansatz(n_qubits = 2*h2.n_orbitals)
#Entangling part of U:
U_ENT = construct_QCC_ansatz(entanglers)

U_QCC = U_MF + U_ENT

E = tq.ExpectationValue(H=H, U=U_QCC)

initial_vals = init_qcc_params(hf_reference, E.extract_variables())

#Minimize wrt the entangler amplitude and MF angles:
result = tq.minimize(objective=E, method="BFGS", initial_values=initial_vals, tol=1.e-6)

print('\nObtained QCC energy ({} entanglers): {}'.format(len(entanglers), result.energy))

Optimizer: <class 'tequila.optimizers.optimizer_scipy.OptimizerSciPy'> 
backend         : qulacs
samples         : None
save_history    : True
noise           : None

Method          : BFGS
Objective       : 1 expectationvalues
gradient        : 18 expectationvalues

active variables : 9

E=-0.70294360  angles= {beta_0: 3.141592653589793, gamma_0: 0.0, beta_1: 3.141592653589793, gamma_1: 0.0, beta_2: 0.0, gamma_2: 0.0, beta_3: 0.0, gamma_3: 0.0, tau_0: 0.0}  samples= None
E=-0.77938639  angles= {beta_0: 3.141592653589793, gamma_0: 0.0, beta_1: 3.141592653589793, gamma_1: 0.0, beta_2: 0.0, gamma_2: 0.0, beta_3: 0.0, gamma_3: 0.0, tau_0: 0.2822100520133972}  samples= None
E=-0.93592356  angles= {beta_0: 3.141592653589793, gamma_0: 0.0, beta_1: 3.141592653589793, gamma_1: 0.0, beta_2: 0.0, gamma_2: 0.0, beta_3: 0.0, gamma_3: 0.0, tau_0: 1.411050260066986}  samples= None
E=-0.93603455  angles= {beta_0: 3.141592653589793, gamma_0: 0.0, beta_1: 3.141592653589793, gamma_1: 0.0, beta_2: 0.0, g

We see that the QCC energy converged to the FCI energy with only a single entangler! 

### H2O in 6-31G basis

Let us move on to the problem of H$_2$O in 6-31G basis using the same active space as the UCCSD example.

In [10]:
xyz_data = get_molecular_data('h2o', geometry=1, xyz_format=True)

basis = '6-31g'
active = {'B1':[0,1], 'A1':[2,3]}
h2o = tq.quantumchemistry.Molecule(geometry=xyz_data, basis_set = basis, active_orbitals = active)
hf_reference = hf_occ(2*h2o.n_orbitals, h2o.n_electrons)


H = h2o.make_hamiltonian()

print("\nHamiltonian has {} terms\n".format(len(H)))

#Define number of entanglers to enter ansatz
n_ents = 6

#Rank entanglers using energy gradient criterion
ranked_entangler_groupings = generate_QCC_gradient_groupings(H.to_openfermion(), 
                                                             2*h2o.n_orbitals, 
                                                             hf_reference, 
                                                             cutoff=threshold)

print('Grouping gradient magnitudes (Grouping : Gradient magnitude):')
for i in range(len(ranked_entangler_groupings)):
    print('{} : {}'.format(i+1,ranked_entangler_groupings[i][1]))

entanglers = get_QCC_entanglers(ranked_entangler_groupings, n_ents, 2*h2o.n_orbitals)

print('\nSelected entanglers:')
for ent in entanglers:
    print(ent)

There are known issues with some psi4 methods and frozen virtual orbitals. Proceed with fingers crossed for hf.

Hamiltonian has 185 terms

Grouping gradient magnitudes (Grouping : Gradient magnitude):
1 : 0.0758
2 : 0.0371
3 : 0.0371
4 : 0.031
5 : 0.0301
6 : 0.0249
7 : 0.0198
8 : 0.0198
9 : 0.0173
10 : 0.0173
11 : 0.0
12 : 0.0

Selected entanglers:
1.0 [X0 Y1 X6 X7]
1.0 [X0 Y3 X5 X6]
1.0 [X1 Y2 X4 X7]
1.0 [X2 Y3 X4 X5]
1.0 [X0 Y1 X4 X5]
1.0 [X2 Y3 X6 X7]


With the 6 chosen entanglers, we can then run the QCC VQE simulation.

In [11]:
#Mean-field part of U (Omega):    
U_MF = construct_QMF_ansatz(n_qubits = 2*h2o.n_orbitals)
#Entangling part of U:
U_ENT = construct_QCC_ansatz(entanglers)

U_QCC = U_MF + U_ENT

E = tq.ExpectationValue(H=H, U=U_QCC)

initial_vals = init_qcc_params(hf_reference, E.extract_variables())

#Minimize wrt the entangler amplitude and MF angles:
result = tq.minimize(objective=E, method="BFGS", initial_values=initial_vals, tol=1.e-4)


print('\nObtained QCC energy ({} entanglers): {}'.format(len(entanglers), result.energy))

Optimizer: <class 'tequila.optimizers.optimizer_scipy.OptimizerSciPy'> 
backend         : qulacs
samples         : None
save_history    : True
noise           : None

Method          : BFGS
Objective       : 1 expectationvalues
gradient        : 44 expectationvalues

active variables : 22

E=-75.98071144  angles= {beta_0: 3.141592653589793, gamma_0: 0.0, beta_1: 3.141592653589793, gamma_1: 0.0, beta_2: 3.141592653589793, gamma_2: 0.0, beta_3: 3.141592653589793, gamma_3: 0.0, beta_4: 0.0, gamma_4: 0.0, beta_5: 0.0, gamma_5: 0.0, beta_6: 0.0, gamma_6: 0.0, beta_7: 0.0, gamma_7: 0.0, tau_0: 0.0, tau_1: 0.0, tau_2: 0.0, tau_3: 0.0, tau_4: 0.0, tau_5: 0.0}  samples= None
E=-75.98783364  angles= {beta_0: 3.141592653589793, gamma_0: 0.0, beta_1: 3.141592653589793, gamma_1: 0.0, beta_2: 3.141592653589793, gamma_2: 0.0, beta_3: 3.141592653589793, gamma_3: 0.0, beta_4: 0.0, gamma_4: 0.0, beta_5: 0.0, gamma_5: 0.0, beta_6: 0.0, gamma_6: 0.0, beta_7: 0.0, gamma_7: 0.0, tau_0: 0.07584381103515625, 

We obtain chemical accuracy for water near equilibrium geometry with only 6 entanglers. The obtained energy is not as accurate as that of UCCSD for this problem, however the QCC optimization may be performed at a fraction of the UCCSD circuit depth. One can also increase the number of entanglers entering the QCC ansatz to increase accuracy. As a final check, one can always run $n$ VQE trials with random initial guesses to test if the optimization fell into a local minimum. ***(Warning: Completing n=10 trials may take a few minutes for this VQE instance).***

In [12]:
n = 10
result = minimize_E_random_guesses(objective=E, method='BFGS', tol=1e-4, n=n)

print('\nObtained QCC energy ({} entanglers): {}'.format(len(entanglers), result))

Optimizer: <class 'tequila.optimizers.optimizer_scipy.OptimizerSciPy'> 
backend         : qulacs
samples         : None
save_history    : True
noise           : None

Method          : BFGS
Objective       : 1 expectationvalues
gradient        : 44 expectationvalues

active variables : 22

E=-74.52072848  angles= {beta_0: 0.4806825781292582, gamma_0: 5.1347789437584765, beta_1: 0.787669258026806, gamma_1: 3.6944385439446377, beta_2: 3.7320356505847245, gamma_2: 3.001129624634042, beta_3: 3.2603022940580177, gamma_3: 0.43067572359283124, beta_4: 0.9358613724219399, gamma_4: 2.613755338008583, beta_5: 3.0598095934308747, gamma_5: 4.141963780036717, beta_6: 1.982377620351075, gamma_6: 5.508030002052578, beta_7: 5.16255602363849, gamma_7: 0.4883635595615649, tau_0: 1.497517704248012, tau_1: 3.0327275269634644, tau_2: 1.0293114142218407, tau_3: 4.290144200855658, tau_4: 3.5474046718157446, tau_5: 5.47074358253356}  samples= None
E=-74.60175665  angles= {beta_0: 0.4790994787640238, gamma_0: 

E=-75.97898465  angles= {beta_0: -0.04794616759597759, gamma_0: 5.6663072060164446, beta_1: 0.02013120692974025, gamma_1: 3.3647623921883536, beta_2: 3.2329629658376673, gamma_2: 3.2021039928196418, beta_3: 3.1403564043994696, gamma_3: 0.3892961203327699, beta_4: 2.937616838912942, gamma_4: 2.0062878761166414, beta_5: 3.199361987897659, gamma_5: 4.151903519871863, beta_6: 0.23095553338643665, gamma_6: 5.632572056761468, beta_7: 6.190825689515336, gamma_7: 1.2663694290587542, tau_0: 3.0508988347251527, tau_1: 3.199388872955985, tau_2: 3.2557454810206368, tau_3: 3.2035279651138775, tau_4: 3.2076035946006227, tau_5: 6.309641065718457}  samples= None
E=-75.98641115  angles= {beta_0: -0.03853447926872665, gamma_0: 5.66962613589394, beta_1: 0.016456940773762422, gamma_1: 3.3636552003317632, beta_2: 3.1136141255047236, gamma_2: 3.2051071434390335, beta_3: 3.122022160527018, gamma_3: 0.3891737309486684, beta_4: 3.0240495710998765, gamma_4: 1.9974970506317897, beta_5: 3.185120392490791, gamma_5

Optimizer: <class 'tequila.optimizers.optimizer_scipy.OptimizerSciPy'> 
backend         : qulacs
samples         : None
save_history    : True
noise           : None

Method          : BFGS
Objective       : 1 expectationvalues
gradient        : 44 expectationvalues

active variables : 22

E=-74.58235915  angles= {beta_0: 2.08428538260868, gamma_0: 3.4365833406642934, beta_1: 2.9362825086085875, gamma_1: 5.23641987366335, beta_2: 5.7051058983367, gamma_2: 3.792188840408101, beta_3: 0.5332118827039052, gamma_3: 5.311975068075997, beta_4: 2.902790587082323, gamma_4: 0.9521505171373335, beta_5: 0.49560343704020865, gamma_5: 3.5058938328221942, beta_6: 0.9313194740711045, gamma_6: 0.7579635741181135, beta_7: 1.209946054884254, gamma_7: 3.59934111019112, tau_0: 4.397069055210107, tau_1: 5.113674424947348, tau_2: 3.7829034992606747, tau_3: 1.5356681921626112, tau_4: 4.735104196162519, tau_5: 6.003004714145949}  samples= None
E=-74.65063112  angles= {beta_0: 2.1411548894446173, gamma_0: 3.461

E=-75.97555668  angles= {beta_0: 3.200709106686107, gamma_0: 4.019128553376254, beta_1: 3.069923037327841, gamma_1: 5.3069270532279, beta_2: 6.198099023755073, gamma_2: 3.713454989198935, beta_3: -0.0041136368058593015, gamma_3: 5.543933079090346, beta_4: 3.2523614536824628, gamma_4: 0.9535939778727547, beta_5: 2.8889279244108774, gamma_5: 3.565016537698671, beta_6: -0.16574777368864652, gamma_6: 1.3380438401266628, beta_7: -0.09372285142909621, gamma_7: 3.703639257461769, tau_0: 3.0291314849334756, tau_1: 3.2363644898120945, tau_2: 3.1610370025729964, tau_3: -0.020352673592206195, tau_4: 6.373973424692108, tau_5: 6.268316238304301}  samples= None
E=-75.98220117  angles= {beta_0: 3.178661518397843, gamma_0: 4.018837356705027, beta_1: 3.0850273875159284, gamma_1: 5.306447461630851, beta_2: 6.200615872264455, gamma_2: 3.7135231337221066, beta_3: -0.010357771409399131, gamma_3: 5.544296162371696, beta_4: 3.2609579552665537, gamma_4: 0.9534042005049179, beta_5: 2.989637091749394, gamma_5: 

E=-75.98858350  angles= {beta_0: 3.141464190981752, gamma_0: 4.018245775754894, beta_1: 3.1419181612118057, gamma_1: 5.305028215268558, beta_2: 6.282423399576017, gamma_2: 3.713390508337824, beta_3: -4.453548455516618e-05, gamma_3: 5.5457647090864555, beta_4: 3.142100341304824, gamma_4: 0.95283021448318, beta_5: 3.141538410132773, gamma_5: 3.5615481098459156, beta_6: 0.0006760271954462866, gamma_6: 1.3397271765798842, beta_7: 0.00014244254882299508, gamma_7: 3.700918634558207, tau_0: 3.0431177890218035, tau_1: 3.2027415488493083, tau_2: 3.2039609607665387, tau_3: 0.048200800051051845, tau_4: 6.324835696719732, tau_5: 6.317490417876993}  samples= None
E=-75.98858364  angles= {beta_0: 3.141496594374001, gamma_0: 4.018265034857533, beta_1: 3.1417896571504085, gamma_1: 5.305028286343221, beta_2: 6.282842134338866, gamma_2: 3.713389353410055, beta_3: -1.2649936098959084e-05, gamma_3: 5.545770706384316, beta_4: 3.1417756741150678, gamma_4: 0.9528297370645339, beta_5: 3.141647084617498, gamma

...

E=-75.51427970  angles= {beta_0: 2.9800940695456313, gamma_0: 5.411515449854248, beta_1: 3.14535134451998, gamma_1: 1.3620011045848786, beta_2: 3.0398920956249373, gamma_2: 2.7388112317905584, beta_3: 6.117563717483666, gamma_3: 6.18793876917594, beta_4: 0.7642388463639502, gamma_4: 5.929803300589341, beta_5: 3.1227545216808563, gamma_5: 4.616869848754978, beta_6: 4.661907726260579, gamma_6: 4.101838495190959, beta_7: 3.182067190786206, gamma_7: 5.878933868452062, tau_0: -0.04689659725377243, tau_1: 3.1145690053730215, tau_2: 3.1097172988044246, tau_3: 3.0553378519002306, tau_4: 3.233172697980213, tau_5: 0.22527286047172784}  samples= None
E=-75.53899419  angles= {beta_0: 3.115433062337295, gamma_0: 5.407796846927801, beta_1: 3.1451124906864085, gamma_1: 1.3601079408394443, beta_2: 3.1112373953409502, gamma_2: 2.745539712790438, beta_3: 6.2690787280357565, gamma_3: 6.186753737617035, beta_4: 0.7940624497755023, gamma_4: 5.934165611788157, beta_5: 3.1529721586498987, gamma_5: 4.61716057

E=-75.94318680  angles= {beta_0: 3.2990253178354916, gamma_0: 5.38599861560804, beta_1: 3.0109133345018604, gamma_1: 1.4365426926772598, beta_2: 3.1301086618291634, gamma_2: 2.809424638462805, beta_3: 9.71695074291136, gamma_3: 6.193695241216448, beta_4: -2.439656277480941, gamma_4: 6.0287119120980615, beta_5: 3.1356752303195736, gamma_5: 4.608752272137124, beta_6: 3.1840743998936, gamma_6: 4.080296767629936, beta_7: 3.045341211786748, gamma_7: 5.86888440514874, tau_0: -0.10857039075400873, tau_1: 3.2159779539036477, tau_2: 3.1490541187289454, tau_3: 3.271576057645186, tau_4: 3.159351624408676, tau_5: -0.003062621910745772}  samples= None
E=-75.95711179  angles= {beta_0: 3.346434449752794, gamma_0: 5.386426291102904, beta_1: 3.0979400368730565, gamma_1: 1.4346816350396385, beta_2: 3.190699050898595, gamma_2: 2.8092945413669015, beta_3: 9.595347100270708, gamma_3: 6.193559641887877, beta_4: -2.5836368307194766, gamma_4: 6.02698003868392, beta_5: 3.1512394750889494, gamma_5: 4.6089647265

E=-75.98858360  angles= {beta_0: 3.141655875298698, gamma_0: 5.3879655087470555, beta_1: 3.141650752744884, gamma_1: 1.4385833525930523, beta_2: 3.141585809484051, gamma_2: 2.8133787649919877, beta_3: 9.424794307804921, gamma_3: 6.194777914164983, beta_4: -3.1416507939598106, gamma_4: 6.035233334384042, beta_5: 3.1425035864216646, gamma_5: 4.607948894403556, beta_6: 3.1416292418177965, gamma_6: 4.076615608247431, beta_7: 3.1416078471267372, gamma_7: 5.880153031121313, tau_0: -0.09852652742188087, tau_1: 3.2031138575152163, tau_2: 3.2039484202976625, tau_3: 3.189627080978266, tau_4: 3.1829422988609624, tau_5: 0.03444515476524817}  samples= None
E=-75.98858367  angles= {beta_0: 3.141648582009734, gamma_0: 5.3879640619507745, beta_1: 3.141636325917501, gamma_1: 1.438579526681327, beta_2: 3.141606337882895, gamma_2: 2.813377440872801, beta_3: 9.424815088950995, gamma_3: 6.194775933632132, beta_4: -3.1415641556652987, gamma_4: 6.035227207078293, beta_5: 3.1417936537788687, gamma_5: 4.607949

E=-75.71876929  angles= {beta_0: 6.405514305432421, gamma_0: 4.868595066280986, beta_1: 3.188935506446573, gamma_1: 2.5294911298284752, beta_2: 6.331285145405222, gamma_2: 2.3220037399268993, beta_3: 3.164031883781722, gamma_3: 2.749733812448088, beta_4: 6.434050527405845, gamma_4: 2.5275384024696166, beta_5: 3.2594248064346716, gamma_5: 1.597311316084773, beta_6: 2.4668948375304782, gamma_6: 1.6585909873074545, beta_7: 3.123006269047579, gamma_7: 5.546170483541752, tau_0: 3.1516906741436963, tau_1: 3.1459326704310726, tau_2: 6.22434726128768, tau_3: 3.248865317503781, tau_4: 3.1890457363790574, tau_5: 6.348624538682734}  samples= None
E=-75.74091650  angles= {beta_0: 6.517277683836146, gamma_0: 4.866403826097496, beta_1: 3.37858490772682, gamma_1: 2.5316441865262953, beta_2: 6.245826068701594, gamma_2: 2.323320486632212, beta_3: 3.300382854422088, gamma_3: 2.749654585081141, beta_4: 6.519838295782718, gamma_4: 2.5260340132761248, beta_5: 3.4102052445257516, gamma_5: 1.5967577579103782

E=-75.98856583  angles= {beta_0: 6.28280409222228, gamma_0: 4.8064809117856, beta_1: 3.14296695691455, gamma_1: 2.625115824782805, beta_2: 6.282424610405237, gamma_2: 2.3156513810636463, beta_3: 3.1328227085990172, gamma_3: 2.7205884244141245, beta_4: 6.290210859988179, gamma_4: 2.4846380907969543, beta_5: 3.1419297838576368, gamma_5: 1.5922362856695298, beta_6: 0.0027119516614066263, gamma_6: 1.6615171038877972, beta_7: 3.138075526621981, gamma_7: 5.534936643072127, tau_0: 3.2403512687991545, tau_1: 3.1999413153002396, tau_2: 6.345967467819918, tau_3: 3.1898493955285057, tau_4: 3.182579555705624, tau_5: 6.3166931430777975}  samples= None
E=-75.98857840  angles= {beta_0: 6.282105032192119, gamma_0: 4.806686165429293, beta_1: 3.141944988589323, gamma_1: 2.6248462520142923, beta_2: 6.281906157378354, gamma_2: 2.3157038297294195, beta_3: 3.139101386565829, gamma_3: 2.720597691455711, beta_4: 6.289155089817879, gamma_4: 2.4847007679668494, beta_5: 3.1413470799914367, gamma_5: 1.59220622240